2015 Day 22

https://adventofcode.com/2015/day/22

In [1]:
import numpy as np
import scipy as sp
import re
import time
import itertools
import string
import collections
import math
import functools
import copy
import operator
import sys
import av
import codecs
import json
import heapq
from PIL import Image, ImageDraw, ImageFont

In [2]:

boss_dam = 8

cost = {"missile" : 53, "drain" :73, "shield" : 113, "poison" : 173, "recharge" : 229}
duration = {"missile" : 0, "drain" : 0, "shield" : 6, "poison" : 6, "recharge" : 5}

start_effect = {"missile" : 0, "drain" : 0, "shield" : 0, "poison" : 0, "recharge" : 0}

start_state = [50, 500, 55, start_effect, 0, 1e5, 0, ""]



In [3]:
def cast_spell(spell, state):
    my_hp, my_mana, boss_hp, old_effect, spent, lowest, depth, battle_string = state
    my_ac = 0

    effect = copy.deepcopy(old_effect)

    battle_string += "\n" + " " * depth + "-- Player Turn --" + "\n"

    battle_string += " " * depth +  "- Player has %d hit points, %d mana" % (my_hp, my_mana) + "\n"
    battle_string += " " * depth + "- Boss has %d hit points" % boss_hp + "\n"

    if part2:
        my_hp -= 1
        battle_string += " " * depth + "Player loses 1 point, bringing hp down to %d" % my_hp + "\n"

        if my_hp <= 0:
            return 1e5, ""

    for i in effect.keys():
        if effect[i] > 0:
            match i:
                case "shield":
                    my_ac = 7
                    effect[i] -= 1
                    battle_string += " " * depth + "shield providing 7 armor; its timer is now %d." % effect[i] +"\n"
                case "poison":
                    boss_hp -= 3
                    effect[i] -= 1
                    battle_string += " " * depth + "Poison deals 3 damage; its timer is now %d." % effect[i]+"\n"
                case "recharge":
                    my_mana += 101
                    effect[i] -= 1
                    battle_string += " " * depth + "Recharge provides 101 mana; its timer is now %d." % effect[i] + "\n"
            if effect[i] == 0:
                battle_string += " " * depth + "%s wears off" % i + "\n" 

    if my_mana < cost[spell]:
        return 1e5, ""

    spent += cost[spell]

    if spent >= lowest:
        return 1e5, ""


    match spell:
        case "missile":
            boss_hp -= 4
            battle_string += " " * depth + "Player casts Magic Missile, dealing 4 damage. Boss at %d" % boss_hp + "\n"
        case "drain":
            boss_hp -= 2
            my_hp += 2
            battle_string += " " * depth + "Player casts Drain, dealing 2 damage and healing for 2. Boss at %d, player at %d" % (boss_hp, my_hp) + "\n"
        case "poison":
            effect[spell] = 6
            battle_string += " " * depth + "Player casts poison." + "\n"
        case "shield":
            battle_string += " " * depth + "Player casts shield." + "\n"
            effect[spell] = 6
        case "recharge":
            battle_string += " " * depth + "Player casts recharge." + "\n"
            effect[spell] = 5

    my_mana -= cost[spell]


    if boss_hp <= 0:
        return spent, battle_string

    battle_string += "\n" + " " * depth + "-- Boss Turn --" + "\n"

    my_ac = 0
    for i in effect.keys():
        if effect[i] > 0:
            match i:
                case "shield":
                    my_ac = 7
                    effect[i] -= 1
                    battle_string += " " * depth + "shield providing 7 armor; its timer is now %d." % effect[i]+ "\n"
                case "poison":
                    boss_hp -= 3
                    effect[i] -= 1
                    battle_string += " " * depth + "Poison deals 3 damage; its timer is now %d.  Boss at %d" % (effect[i], boss_hp) + "\n"
                case "recharge":
                    my_mana += 101
                    effect[i] -= 1
                    battle_string += " " * depth + "Recharge provides 101 mana; its timer is now %d." % effect[i]+ "\n"
            if effect[i] == 0:
                battle_string += " " * depth + "%s wears off" % i + "\n"
                
    if boss_hp <= 0:
        return spent, battle_string

    dam = max(boss_dam - my_ac, 1)
    my_hp -= dam
    battle_string += " " * depth + "Boss deals %d damage.  Player at %d" % (dam, my_hp) + "\n"


    if my_hp <= 0:
        return 1e5, ""

    next_mana = my_mana + (101 if effect["recharge"] > 0 else 0)

    lo_str = ""

    for i in cost.keys():
        if next_mana >= cost[i]:
            next_state = [my_hp, my_mana, boss_hp, effect, spent, lowest, depth+1, battle_string]
            child, child_str = cast_spell(i, next_state)
            if child < lowest:
                lowest = child 
                lo_str = child_str

    return lowest, lo_str


In [10]:


def attack_bfs(part2 = False):

    queue = []
    tiebreaker = itertools.count()
    queue.append([0, next(tiebreaker), "", start_state])
    lowest = 1e5

    
#    print("q:", queue)
    while queue:
        foo, _, spell, state = heapq.heappop(queue)

#        print("spell:", spell)
#        print(state)
        my_hp, my_mana, boss_hp, old_effect, spent, lowest, depth, battle_string = state
        my_ac = 0

        effect = copy.deepcopy(old_effect)
            
        if foo > 0:

#            print(foo)

#            print(effect)
            battle_string += "\n" + " " * depth + "-- Player Turn --" + "\n"

            battle_string += " " * depth +  "- Player has %d hit points, %d mana" % (my_hp, my_mana) + "\n"
            battle_string += " " * depth + "- Boss has %d hit points" % boss_hp + "\n"

            if part2:
                my_hp -= 1
                battle_string += " " * depth + "Player loses 1 point, bringing hp down to %d" % my_hp + "\n"

                if my_hp <= 0:
                    continue

            for i in effect.keys():
                if effect[i] > 0:
                    match i:
                        case "shield":
                            my_ac = 7
                            effect[i] -= 1
                            battle_string += " " * depth + "shield providing 7 armor; its timer is now %d." % effect[i] +"\n"
                        case "poison":
                            boss_hp -= 3
                            effect[i] -= 1
                            battle_string += " " * depth + "Poison deals 3 damage; its timer is now %d." % effect[i]+"\n"
                        case "recharge":
                            my_mana += 101
                            effect[i] -= 1
                            battle_string += " " * depth + "Recharge provides 101 mana; its timer is now %d." % effect[i] + "\n"
                    if effect[i] == 0:
                        battle_string += " " * depth + "%s wears off" % i + "\n" 

            if my_mana < cost[spell]:
                continue

            spent += cost[spell]

            if spent >= lowest:
                continue


            match spell:
                case "missile":
                    boss_hp -= 4
                    battle_string += " " * depth + "Player casts Magic Missile, dealing 4 damage. Boss at %d" % boss_hp + "\n"
                case "drain":
                    boss_hp -= 2
                    my_hp += 2
                    battle_string += " " * depth + "Player casts Drain, dealing 2 damage and healing for 2. Boss at %d, player at %d" % (boss_hp, my_hp) + "\n"
                case "poison":
                    effect[spell] = 6
                    battle_string += " " * depth + "Player casts poison." + "\n"
                case "shield":
                    battle_string += " " * depth + "Player casts shield." + "\n"
                    effect[spell] = 6
                case "recharge":
                    battle_string += " " * depth + "Player casts recharge." + "\n"
                    effect[spell] = 5

            my_mana -= cost[spell]


            if boss_hp <= 0:
                return spent, battle_string


            battle_string += "\n" + " " * depth + "-- Boss Turn --" + "\n"

            my_ac = 0
            for i in effect.keys():
                if effect[i] > 0:
                    match i:
                        case "shield":
                            my_ac = 7
                            effect[i] -= 1
                            battle_string += " " * depth + "shield providing 7 armor; its timer is now %d." % effect[i]+ "\n"
                        case "poison":
                            boss_hp -= 3
                            effect[i] -= 1
                            battle_string += " " * depth + "Poison deals 3 damage; its timer is now %d.  Boss at %d" % (effect[i], boss_hp) + "\n"
                        case "recharge":
                            my_mana += 101
                            effect[i] -= 1
                            battle_string += " " * depth + "Recharge provides 101 mana; its timer is now %d." % effect[i]+ "\n"
                    if effect[i] == 0:
                        battle_string += " " * depth + "%s wears off" % i + "\n"
                        
            if boss_hp <= 0:
                return spent, battle_string
 

            dam = max(boss_dam - my_ac, 1)
            my_hp -= dam
            battle_string += " " * depth + "Boss deals %d damage.  Player at %d" % (dam, my_hp) + "\n"


            if my_hp <= 0:
                continue

        next_mana = my_mana + (101 if effect["recharge"] > 0 else 0)

        for i in cost.keys():
            if next_mana >= cost[i]:
                next_state = [my_hp, my_mana, boss_hp, effect, spent, lowest, depth+1, battle_string]
#                print("push:", i, next_state)
                heapq.heappush(queue, (spent + cost[i], next(tiebreaker), i, next_state))


    return 1e5, ""

In [5]:

def attack(state):
    my_hp, my_mana, boss_hp, effect, spent, lowest, depth, battle_string = state    
    for i in cost.keys():

        next_state = [my_hp, my_mana, boss_hp, effect, spent, lowest, depth + 1, ""]

#        print("casting %s" %i, next_state)
        child, child_str = cast_spell(i, next_state)
        if child < lowest:
            lowest = child 
            lo_str = child_str

    return lowest, lo_str


In [8]:

start_time = time.time()

lowest, lo_str = attack_bfs()


print(lowest)

print("execution time (in ms): ",(time.time()-start_time)*1000) 


start_time = time.time()

lowest, lo_str = attack_bfs(True)


print(lowest)

print("execution time (in ms): ",(time.time()-start_time)*1000) 


953
execution time (in ms):  3482.1815490722656
953
execution time (in ms):  0.0
